In [1]:
# final run
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)

    root.mainloop()
    root.destroy()  # ✅ Ensure window is closed

def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()  # ✅ Fully close the window
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()  # ✅ Cleanup after loop ends
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()  # ✅ Fully close after use
    return result

def split_excel_preserving_format():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)

            # Remove excluded sheets
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Set as the only selected tab
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                # Clear previous data except header
                max_row = ws.max_row
                if max_row > 1:
                    ws.delete_rows(2, max_row - 1)

                # Write filtered data
                filtered_df = df[df[split_column] == val]
                for i, row in enumerate(filtered_df.values, start=2):
                    for j, value in enumerate(row, start=1):
                        ws.cell(row=i, column=j).value = value

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")
#                 messagebox.showinfo("Done", f"Files split and formatting preserved!\nSaved to:\n{output_dir}")

        # Force close any leftover windows
        for widget in tk._default_root.winfo_children():
            widget.destroy()
        if tk._default_root:
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format()


✅ Saved: C:/Users/KS\Split_Formatted_Excel\London.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Madrid.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Lisbon.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Berlin.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Paris.xlsx


In [3]:
# format preservation
# final run with formatting & formula preservation
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook
from copy import copy

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

# 👉 Helper to copy formatting & formula
def copy_row(ws_from, ws_to, row_idx_from, row_idx_to):
    for col_idx, cell in enumerate(ws_from[row_idx_from], start=1):
        new_cell = ws_to.cell(row=row_idx_to, column=col_idx)
        new_cell.value = cell.value
        new_cell.data_type = cell.data_type

        if cell.has_style:
            new_cell.font = copy(cell.font)
            new_cell.border = copy(cell.border)
            new_cell.fill = copy(cell.fill)
            new_cell.number_format = copy(cell.number_format)
            new_cell.protection = copy(cell.protection)
            new_cell.alignment = copy(cell.alignment)

        if cell.hyperlink:
            new_cell.hyperlink = copy(cell.hyperlink)

        if cell.comment:
            new_cell.comment = copy(cell.comment)

def split_excel_preserving_format():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            if sheet_to_split not in wb.sheetnames:
                continue

            # Remove excluded sheets
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            ws = wb[sheet_to_split]

            # Set as active tab
            for w in wb.worksheets:
                w.sheet_view.tabSelected = False
            ws.sheet_view.tabSelected = True
            wb.active = wb.sheetnames.index(sheet_to_split)

            # Copy original sheet (used to preserve formulas/formatting)
            original_ws = load_workbook(file_path, data_only=False)[sheet_to_split]

            # Clear old data below header
            max_row = ws.max_row
            if max_row > 1:
                ws.delete_rows(2, max_row - 1)

            filtered_df = df[df[split_column] == val]

            for idx, (_, row) in enumerate(filtered_df.iterrows(), start=2):
                source_excel_row_index = row.name + 2  # Adjust for Excel's row numbers
                copy_row(original_ws, ws, source_excel_row_index, idx)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")

        for widget in tk._default_root.winfo_children():
            widget.destroy()
        if tk._default_root:
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format()


✅ Saved: C:/Users/KS\Split_Formatted_Excel\Online .xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\In-store .xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Drive-thru .xlsx


In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.worksheet.worksheet import Worksheet

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

def copy_row(source_ws: Worksheet, dest_ws: Worksheet, source_row: int, dest_row: int):
    for col in range(1, source_ws.max_column + 1):
        source_cell = source_ws.cell(row=source_row, column=col)
        dest_cell = dest_ws.cell(row=dest_row, column=col)

        dest_cell.value = source_cell.value
        dest_cell.data_type = source_cell.data_type
        dest_cell.number_format = source_cell.number_format
        dest_cell.font = source_cell.font
        dest_cell.border = source_cell.border
        dest_cell.fill = source_cell.fill
        dest_cell.alignment = source_cell.alignment

        if source_cell.has_formula:
            dest_cell.value = f"={source_cell.value}"

def split_excel_preserving_format():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            original_wb = load_workbook(file_path)
            original_ws = original_wb[sheet_to_split]

            # Remove excluded sheets
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Set tab active
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                # Clear previous data except header
                max_row = ws.max_row
                if max_row > 1:
                    ws.delete_rows(2, max_row - 1)

                # Filtered data
                filtered_df = df[df[split_column] == val]

                # ✅ Copy matching rows with formatting and formulas
                for i in range(len(filtered_df)):
                    excel_source_row_index = filtered_df.index[i] + 2  # original Excel row (1-based + header)
                    target_row_index = i + 2  # destination row in new sheet
                    copy_row(original_ws, ws, excel_source_row_index, target_row_index)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")

        # Close any leftover windows
        for widget in tk._default_root.winfo_children():
            widget.destroy()
        if tk._default_root:
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format()


In [1]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xlsm")])
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")
    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result

def split_excel_preserving_everything():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, dtype=str)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            if sheet_to_split not in wb.sheetnames:
                continue

            # Remove excluded sheets
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            ws = wb[sheet_to_split]
            headers = [cell.value for cell in ws[1]]

            # Create a mapping of column name to column number
            col_map = {cell.value: idx+1 for idx, cell in enumerate(ws[1])}
            filter_col_idx = col_map.get(split_column)
            if not filter_col_idx:
                continue

            rows_to_keep = [1]  # Always keep header
            for i in range(2, ws.max_row + 1):
                cell_val = ws.cell(i, filter_col_idx).value
                if str(cell_val).strip() == str(val).strip():
                    rows_to_keep.append(i)

            # Delete rows not in the filtered list
            all_rows = list(range(2, ws.max_row + 1))
            delete_rows = [i for i in all_rows if i not in rows_to_keep]

            for row in reversed(delete_rows):
                ws.delete_rows(row, 1)

            # Ensure it's the active selected sheet
            for w in wb.worksheets:
                w.sheet_view.tabSelected = False
            ws.sheet_view.tabSelected = True
            wb.active = wb.sheetnames.index(sheet_to_split)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split successfully!\nSaved to:\n{output_dir}")

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_everything()


✅ Saved: C:/Users/KS\Split_Formatted_Excel\London.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Madrid.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Lisbon.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Berlin.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Paris.xlsx


In [5]:
#  Save formulas too
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Select this tab
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                # Backup formulas from first data row
                start_row = 2
                formulas = []
                for cell in ws[start_row]:
                    if cell.data_type == 'f':
                        formulas.append(f"={cell.value}")
                    else:
                        formulas.append(cell.value)

                # Clear previous data rows
                max_row = ws.max_row
                if max_row > 1:
                    ws.delete_rows(2, max_row - 1)

                # Filter data
                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        ws.cell(row=start_row + i, column=j, value=value)

                    # Apply formulas row-wise
                    for j, orig_formula in enumerate(formulas, start=1):
                        if isinstance(orig_formula, str) and orig_formula.startswith("="):
                            origin_cell = f"{get_column_letter(j)}{start_row}"
                            target_cell = f"{get_column_letter(j)}{start_row + i}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            ws.cell(row=start_row + i, column=j, value=translated)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting + formulas preserved!\nSaved to:\n{output_dir}")
        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()
    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


✅ Saved: C:/Users/KS\Split_Formatted_Excel\1.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\2.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\3.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\4.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\5.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\6.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\7.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\8.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\9.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\10.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\11.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\12.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\13.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\14.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\15.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\16.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\17.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\18.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\19.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Exc

✅ Saved: C:/Users/KS\Split_Formatted_Excel\161.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\162.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\163.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\164.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\165.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\166.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\167.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\168.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\169.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\170.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\171.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\172.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\173.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\174.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\175.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\176.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\177.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\178.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\179.xlsx
✅ Saved: C:/

KeyboardInterrupt: 

In [3]:
# Final Code with formatting 

import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from tkinter import simpledialog  # Add this if you need askstring
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=1"0)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()
    root.attributes("-topmost", True)
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")
    root.attributes("-topmost", True)

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    root.attributes("-topmost", True)

    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

def split_excel_preserving_format_and_formulas():
    show_welcome()
    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", msg)
        log_error(msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                start_row = 2
                formulas = []
                for cell in ws[start_row]:
                    if cell.data_type == 'f':
                        formulas.append(f"={cell.value}")
                    else:
                        formulas.append(cell.value)

                max_row = ws.max_row
                if max_row > 1:
                    ws.delete_rows(2, max_row - 1)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        ws.cell(row=start_row + i, column=j, value=value)

                    for j, orig_formula in enumerate(formulas, start=1):
                        if isinstance(orig_formula, str) and orig_formula.startswith("="):
                            origin_cell = f"{get_column_letter(j)}{start_row}"
                            target_cell = f"{get_column_letter(j)}{start_row + i}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            ws.cell(row=start_row + i, column=j, value=translated)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split with formatting + formulas preserved!\nSaved to:\n{output_dir}")
        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()
    except Exception as e:
        msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", msg)
        log_error(msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


✅ Saved: C:/Users/KS\Split_Formatted_Excel\1.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\2.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\3.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\4.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\5.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\6.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\7.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\8.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\9.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\10.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\11.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\12.xlsx


KeyboardInterrupt: 

In [5]:


import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
from copy import copy

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=10)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Select this tab
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                # Backup template row 2
                start_row = 2
                template_row = list(ws.iter_rows(min_row=start_row, max_row=start_row))[0]
                template_data = []
                for cell in template_row:
                    template_data.append({
                        "value": cell.value,
                        "is_formula": cell.data_type == "f",
                        "style": copy(cell._style),
                        "number_format": cell.number_format
                    })

                # Clear all data rows after header
                if ws.max_row > 1:
                    ws.delete_rows(start_row, ws.max_row - 1)

                # Filter data
                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, (value, cell_template) in enumerate(zip(row, template_data), start=1):
                        cell = ws.cell(row=start_row + i, column=j)
                        cell.value = value
                        cell._style = copy(cell_template["style"])
                        cell.number_format = cell_template["number_format"]

                    for j, cell_template in enumerate(template_data, start=1):
                        if cell_template["is_formula"]:
                            origin = f"{get_column_letter(j)}{start_row}"
                            target = f"{get_column_letter(j)}{start_row + i}"
                            translated_formula = Translator("=" + str(cell_template["value"]), origin=origin).translate_formula(target)
                            cell = ws.cell(row=start_row + i, column=j)
                            cell.value = translated_formula
                            cell._style = copy(cell_template["style"])
                            cell.number_format = cell_template["number_format"]

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split with formatting + formulas!\nSaved to:\n{output_dir}")
        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()
    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


✅ Saved: C:/Users/KS\Split_Formatted_Excel\London.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Madrid.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Lisbon.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Berlin.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\Paris.xlsx
